## Практична робота 3
### Багатокласова класифікація з використанням бібліотеки глибокого навчання Keras

Виконав студент групи ШІДМ-51 Тертишний В.Ю.

### Виконані завдання:

1. **Дослідження впливу кількості нейронів:**
   - Протестовано моделі з 16, 32 та 34 нейронами в прихованому шарі
   - Виявлено, що збільшення кількості нейронів впливає на точність класифікації

2. **Експерименти з архітектурою мережі:**
   - Додано моделі з різною кількістю прихованих шарів (1, 2 та 4 шари)
   - Протестовано різні конфігурації нейронів у шарах
   - Додано регуляризацію та Dropout для запобігання перенавчання

3. **Реалізація функціонального API:**
   - Створено модель з використанням функціонального API Keras
   - Продемонстровано гнучкість такого підходу при створенні складних архітектур

### Висновки:
Проведені експерименти показали, що архітектура нейронної мережі суттєво впливає на якість класифікації. Використання регуляризації та Dropout допомогло уникнути перенавчання моделі.

In [19]:
import pandas
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [20]:
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [21]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

In [30]:
# define baseline_model
def baseline_model():
# create model
    model = Sequential()

    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam',
    metrics=['accuracy'])
    return model

model = baseline_model()
# Adam optimizer with learning rate of 0.001
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
""" print('Neural Network Model Summary: ')
print(model.summary()) """
train_x, test_x, train_y, test_y = train_test_split(X, dummy_y, test_size=0.2, random_state=42)
# Train the model
print('Size of train_x:', train_x.shape)
print('Size of train_y:', train_y.shape)
print('Size of test_x:', test_x.shape)
print('Size of test_y:', test_y.shape)
model.fit(train_x, train_y, verbose=0, batch_size=5, epochs=200)
# Test on unseen data
results = model.evaluate(test_x, test_y)
print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Size of train_x: (120, 4)
Size of train_y: (120, 3)
Size of test_x: (30, 4)
Size of test_y: (30, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 1.0000 - loss: 0.1236
Final test set loss: 0.123580
Final test set accuracy: 1.000000


In [23]:
# 1. Test different neuron counts
def create_model_variant(hidden_neurons):
    model = Sequential()
    model.add(Dense(hidden_neurons, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
# Test models with 16, 32, and 34 neurons
for neurons in [16, 32, 34]:
    print(f"\nTraining model with {neurons} neurons")
    model = create_model_variant(neurons)
    history = model.fit(train_x, train_y, verbose=2, batch_size=5, epochs=200)
    results = model.evaluate(test_x, test_y)
    print(f'Test accuracy: {results[1]:.4f}')


Training model with 16 neurons
Epoch 1/200
24/24 - 0s - 12ms/step - accuracy: 0.3250 - loss: 2.1664
Epoch 2/200
24/24 - 0s - 1ms/step - accuracy: 0.3250 - loss: 1.6735
Epoch 3/200
24/24 - 0s - 1ms/step - accuracy: 0.3250 - loss: 1.3503
Epoch 4/200
24/24 - 0s - 1ms/step - accuracy: 0.3250 - loss: 1.1376
Epoch 5/200
24/24 - 0s - 1ms/step - accuracy: 0.4000 - loss: 1.0365
Epoch 6/200
24/24 - 0s - 1ms/step - accuracy: 0.4000 - loss: 0.9400
Epoch 7/200
24/24 - 0s - 1ms/step - accuracy: 0.6833 - loss: 0.8671
Epoch 8/200
24/24 - 0s - 1ms/step - accuracy: 0.8583 - loss: 0.8006
Epoch 9/200
24/24 - 0s - 1ms/step - accuracy: 0.9000 - loss: 0.7427
Epoch 10/200
24/24 - 0s - 1ms/step - accuracy: 0.8833 - loss: 0.6911
Epoch 11/200
24/24 - 0s - 1ms/step - accuracy: 0.8917 - loss: 0.6452
Epoch 12/200
24/24 - 0s - 1ms/step - accuracy: 0.8917 - loss: 0.6054
Epoch 13/200
24/24 - 0s - 1ms/step - accuracy: 0.8917 - loss: 0.5707
Epoch 14/200
24/24 - 0s - 1ms/step - accuracy: 0.8833 - loss: 0.5411
Epoch 15/2

In [25]:
# 2. Multiple hidden layers
def create_deep_model(layer_sizes):
    model = Sequential()
    model.add(Dense(layer_sizes[0], input_dim=4, activation='relu'))
    for size in layer_sizes[1:]:
        model.add(Dense(size, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
# Test different architectures
architectures = [
    [16, 16],           # 2 layers
    [32, 16, 8],        # 3 layers
    [32, 16, 8, 4]      # 4 layers
]

for layers in architectures:
    print(f"\nTraining model with architecture: {layers}")
    model = create_deep_model(layers)
    history = model.fit(train_x, train_y, verbose=0, batch_size=5, epochs=200)
    results = model.evaluate(test_x, test_y)
    print(f'Test accuracy: {results[1]:.4f}')


Training model with architecture: [16, 16]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 1.0000 - loss: 0.0859
Test accuracy: 1.0000

Training model with architecture: [32, 16, 8]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.9667 - loss: 0.0737
Test accuracy: 0.9667

Training model with architecture: [32, 16, 8, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 1.0000 - loss: 0.0557
Test accuracy: 1.0000


In [29]:
# 3. Functional API version
from tensorflow.keras import Model, Input

def create_functional_model(layer_sizes):
    inputs = Input(shape=(4,))
    x = Dense(layer_sizes[0], activation='relu')(inputs)
    for size in layer_sizes[1:]:
        x = Dense(size, activation='relu')(x)
    outputs = Dense(3, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Test functional API model
functional_model = create_functional_model([32, 16, 8])
history = functional_model.fit(train_x, train_y, verbose=0, batch_size=5, epochs=200)
results = functional_model.evaluate(test_x, test_y)
print(f'Functional API model test accuracy: {results[1]:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 1.0000 - loss: 0.0714
Functional API model test accuracy: 1.0000
